# Consolidação de Arquivos - Detalhes

In [1]:
# "latin1", "iso-8859-1", ou "cp1252",

import re
import os
from pathlib import Path

import duckdb
import openpyxl
import pandas as pd
from pandas import DataFrame

In [2]:
def _importar_arquivo(arquivo: str, aba: str=None, encoding: str ="latin1") -> DataFrame:
    try:
        if arquivo.endswith(".xlsx") or arquivo.endswith(".xls"):
            df = pd.read_excel(arquivo, engine="openpyxl", sheet_name=aba, dtype="str")
        elif arquivo.endswith(".csv"):
            df = pd.read_csv(arquivo, encoding=encoding, delimiter=";", dtype="str")
        elif arquivo.endswith(".txt"):
            df = pd.read_csv(arquivo, encoding=encoding, delimiter=";", dtype="str")
        elif arquivo.endswith(".json"):
            df = pd.read_json(arquivo)
        else:
            raise ValueError("Formato de arquivo não suportado")
        return df
    except Exception as erro:
        raise ValueError(f"Erro: {erro}")

In [3]:
def amostra_atributo(atributo: str, dataframe: DataFrame, num_amostras:int=2) -> str:
    try:
        query = f"""
        SELECT {atributo}
        FROM dataframe WHERE {atributo}
        IS NOT NULL
        LIMIT {num_amostras}
        """
        resultado_query = (
            [
                str(item[0])
                .replace(",", "")
                .strip()
                for item
                in duckdb
                .sql(query)
                .fetchall()
            ]
        )
        if len(resultado_query) == 0:
            resultado_query = ["SEM AMOSTRA"]
        return ";".join(resultado_query)
    except Exception as _:
        return "SEM AMOSTRA" 

In [4]:
def detalhes_arquivos(lista_arquivos: list) -> DataFrame:
    try:
        lista_temp = []
        for item in lista_arquivos:
            caminho = os.path.relpath(item)
            arquivo = f"{Path(os.path.abspath(item)).stem}{Path(os.path.abspath(item)).suffix}" 
            extensao = Path(os.path.abspath(item)).suffix
            try:
                if item.endswith(".xlsx") or item.endswith(".xls"): 
                    abas_planilha = pd.ExcelFile(item).sheet_names

                    for aba in abas_planilha:
                        df = _importar_arquivo(item, aba)
                        colunas = [coluna.upper() for coluna in df.columns.tolist()]
                        quantidades = df.count().tolist()

                        for coluna, quantidade in zip(colunas, quantidades):
                            amostra = amostra_atributo(coluna, df)
                            lista_temp.append(
                                {   
                                    "caminho": caminho,
                                    "arquivo": arquivo,
                                    "coluna": coluna.strip(),
                                    "extensao": extensao,
                                    "amostra": amostra,
                                    "quantidade": quantidade,
                                    "aba": aba,
                                    "observacoes": "",
                                    "status": "OK"
                                    }
                            )
                else:
                    df = _importar_arquivo(item)
                    colunas = [coluna.upper() for coluna in df.columns.tolist()]
                    quantidades = df.count().tolist()
                    for coluna, quantidade in zip(colunas, quantidades):
                        amostra = amostra_atributo(coluna, df)
                        lista_temp.append(
                            {   
                                "caminho": caminho,
                                "arquivo": arquivo,
                                "coluna": coluna.strip(),
                                "extensao": extensao,
                                "amostra": amostra,
                                "quantidade": quantidade,
                                "aba": "",
                                "observacoes": "",
                                "status": "OK"
                                }
                        )
            except Exception as erro:
                lista_temp.append(
                    {
                        "caminho": caminho,
                        "arquivo": arquivo,
                        "coluna": "",
                        "extensao": extensao,
                        "amostra": "",
                        "quantidade": "",
                        "aba": "",                        
                        "observacoes": f"{erro}",
                        "status": "ERRO"
                        }
                )
        return pd.DataFrame(lista_temp)
    except Exception as erro:
        raise ValueError(f"Erro na geração dos detalhes. Erro: {erro}")

### Importando a Lista de arquivos a partir dos Metadados

In [5]:
metadados_arquivos = pd.read_csv("../resultados/metadados/consolidacao_metadados.csv", sep=";", dtype="str")

In [6]:
metadados_arquivos

,caminho,arquivo,extensao,modo,inode,dispositivo,links,uid,gid,tamanho,ultimo_acesso,ultima_modificacao,criacao
0,../arquivos,informacao-cargo.txt,txt,33204,13799851,66307,1,1001,1001,77,06/07/2024 17:22:31,05/07/2024 04:40:26,05/07/2024 04:46:46
1,../arquivos,informacao-projeto.xlsx,xlsx,33204,15230982,66307,1,1001,1001,6704,07/07/2024 04:35:56,07/07/2024 04:35:56,07/07/2024 04:35:56
2,../arquivos,informacao-empresas.csv,csv,33204,13799852,66307,1,1001,1001,84,06/07/2024 17:12:44,05/07/2024 04:40:28,05/07/2024 04:44:54
3,../arquivos,informacao-erro.jpg,jpg,33204,15230954,66307,1,1001,1001,0,06/07/2024 17:29:31,06/07/2024 17:29:31,06/07/2024 17:29:31
4,../arquivos/dir2,informacao-endereco.json,json,33204,13799848,66307,1,1001,1001,231,06/07/2024 17:27:13,05/07/2024 04:40:20,06/07/2024 16:17:58
5,../arquivos/dir2,informacoes-contatos.txt,txt,33204,13799850,66307,1,1001,1001,144,06/07/2024 17:12:44,05/07/2024 04:40:24,06/07/2024 16:18:54
6,../arquivos/dir2/dir2.1,informacao-salario.csv,csv,33204,13799853,66307,1,1001,1001,69,06/07/2024 17:23:06,06/07/2024 17:22:59,06/07/2024 17:22:59
7,../arquivos/dir1,informacao-erro.png,png,33204,15226559,66307,1,1001,1001,0,06/07/2024 17:29:13,06/07/2024 17:29:12,06/07/2024 17:29:12
8,../arquivos/dir1,informacao-departamento.xlsx,xlsx,33204,13799854,66307,1,1001,1001,5517,06/07/2024 17:08:20,05/07/2024 04:40:32,06/07/2024 16:17:45
9,../arquivos/dir1,informacao-pessoa.json,json,33204,13799849,66307,1,1001,1001,174,06/07/2024 17:27:13,05/07/2024 04:40:22,06/07/2024 16:18:56


In [7]:
lista_arquivos = (
    [
        f"{caminho}/{arquivo}"
        for caminho, arquivo
        in zip(metadados_arquivos["caminho"], metadados_arquivos["arquivo"])
    ]
)

In [8]:
lista_arquivos

['../arquivos/informacao-cargo.txt',
 '../arquivos/informacao-projeto.xlsx',
 '../arquivos/informacao-empresas.csv',
 '../arquivos/informacao-erro.jpg',
 '../arquivos/dir2/informacao-endereco.json',
 '../arquivos/dir2/informacoes-contatos.txt',
 '../arquivos/dir2/dir2.1/informacao-salario.csv',
 '../arquivos/dir1/informacao-erro.png',
 '../arquivos/dir1/informacao-departamento.xlsx',
 '../arquivos/dir1/informacao-pessoa.json']

In [9]:
df_detalhes_arquivos = detalhes_arquivos(lista_arquivos)

In [11]:
df_detalhes_arquivos.sort_values(by=["caminho","arquivo","aba","coluna"])

,caminho,arquivo,coluna,extensao,amostra,quantidade,aba,observacoes,status
37,../arquivos/dir1/informacao-departamento.xlsx,informacao-departamento.xlsx,DEPARTAMENTO,.xlsx,RH;TI,3,Sheet1,,OK
34,../arquivos/dir1/informacao-departamento.xlsx,informacao-departamento.xlsx,ID,.xlsx,1;2,3,Sheet1,,OK
36,../arquivos/dir1/informacao-departamento.xlsx,informacao-departamento.xlsx,IDADE,.xlsx,30;25,3,Sheet1,,OK
35,../arquivos/dir1/informacao-departamento.xlsx,informacao-departamento.xlsx,NOME,.xlsx,Ana;Bruno,3,Sheet1,,OK
33,../arquivos/dir1/informacao-erro.png,informacao-erro.png,,.png,,,,Erro: Formato de arquivo não suportado,ERRO
41,../arquivos/dir1/informacao-pessoa.json,informacao-pessoa.json,CPF,.json,111.111.111-11;222.222.222-22,3,,,OK
38,../arquivos/dir1/informacao-pessoa.json,informacao-pessoa.json,ID,.json,1;2,3,,,OK
40,../arquivos/dir1/informacao-pessoa.json,informacao-pessoa.json,IDADE,.json,30;25,3,,,OK
39,../arquivos/dir1/informacao-pessoa.json,informacao-pessoa.json,NOME,.json,Ana;Bruno,3,,,OK
29,../arquivos/dir2/dir2.1/informacao-salario.csv,informacao-salario.csv,ID,.csv,1;2,3,,,OK


In [13]:
(
    df_detalhes_arquivos.sort_values(by=["caminho","arquivo","aba","coluna"])
    .to_csv("../resultados/detalhes/consolidacao_detalhes.csv", index=False, sep=";")
)